## Data preprocessing

In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# === Load and preprocess energy data ===
df = pd.read_csv("mm79158.csv", sep=';', decimal=',')
df['ts'] = pd.to_datetime(df['ts'])
df.set_index('ts', inplace=True)

# Resample to 30-minute intervals and interpolate
df_resampled = df.resample('30min').mean()
df_resampled['vrednost'] = df_resampled['vrednost'].interpolate(method='time')

# Extract time-based features
df_resampled['hour_48'] = df_resampled.index.map(lambda x: (x.hour * 2 + x.minute // 30) + 1)
df_resampled['day_of_week'] = df_resampled.index.dayofweek

# === Load holidays and tag rows ===
holidays = pd.read_csv("slovenian_holidays_2016_2018.csv")
holidays['holiday_date'] = pd.to_datetime(holidays['holiday_date'])
holiday_set = set(holidays['holiday_date'].dt.normalize())
df_resampled['is_holiday'] = df_resampled.index.normalize().isin(holiday_set).astype(int)

# === One-hot encode hour_48, day_of_week, and is_holiday ===
encoder = OneHotEncoder(sparse_output=False)
encoded = encoder.fit_transform(df_resampled[['hour_48', 'day_of_week', 'is_holiday']])
encoded_df = pd.DataFrame(
    encoded,
    columns=encoder.get_feature_names_out(['hour_48', 'day_of_week', 'is_holiday']),
    index=df_resampled.index
)

# === Combine with original value and sort
df_final = pd.concat([df_resampled[['vrednost']], encoded_df], axis=1)
df_final = df_final.sort_index(ascending=True)
df = df_final.copy()


In [2]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def prepare_sequences(df, input_window=48, forecast_horizon=1, val_ratio=0.1, test_ratio=0.1):
    """
    Prepares sliding window sequences for multistep forecasting and applies MinMax scaling to 'vrednost' safely.

    Parameters:
    - df: DataFrame with 'vrednost' as the first column and features after
    - input_window: number of timesteps in the input window
    - forecast_horizon: number of timesteps to predict
    - val_ratio: validation size ratio
    - test_ratio: test size ratio

    Returns:
    - X_train, y_train, X_val, y_val, X_test, y_test, vrednost_scaler
    """
    df = df.copy()  # avoid modifying original

    # === Step 1: Determine sizes
    total_len = len(df) - input_window - forecast_horizon + 1
    val_size = int(total_len * val_ratio)
    test_size = int(total_len * test_ratio)
    train_size = total_len - val_size - test_size

    # Find raw index range for safe fitting
    fit_end_idx = train_size + input_window

    # === Step 2: Fit MinMaxScaler on 'vrednost' using only training range
    # Fit scaler on training data only
    vrednost_scaler = MinMaxScaler()
    df.loc[:df.index[fit_end_idx - 1], 'vrednost'] = vrednost_scaler.fit_transform(
        df.loc[:df.index[fit_end_idx - 1], ['vrednost']]
    ).ravel()

    # Transform validation + test data
    df.loc[df.index[fit_end_idx:], 'vrednost'] = vrednost_scaler.transform(
        df.loc[df.index[fit_end_idx:], ['vrednost']]
    ).ravel()


    # === Step 4: Generate sequences
    values = df.values
    X, y = [], []
    for i in range(total_len):
        X.append(values[i:i + input_window])
        y.append(values[i + input_window:i + input_window + forecast_horizon, 0])  # target = 'vrednost'

    X = np.array(X)
    y = np.array(y)

    if forecast_horizon == 1:
        y = y.reshape(-1, 1)

    # === Step 5: Final chronological split
    X_train, y_train = X[:train_size], y[:train_size]
    X_val, y_val = X[train_size:train_size + val_size], y[train_size:train_size + val_size]
    X_test, y_test = X[train_size + val_size:], y[train_size + val_size:]

    return X_train, y_train, X_val, y_val, X_test, y_test, vrednost_scaler


## Model creation

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dropout, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanAbsoluteError

def build_cnn_lstm_model(input_shape, output_size=1):
    """
    Builds a CNN-LSTM model with up to 3 MaxPooling layers.
    The number of pooling layers is reduced based on input sequence length
    to avoid collapsing the sequence dimension.

    Parameters:
    - input_shape: (timesteps, features)
    - output_size: number of forecast steps (e.g., 1, 2, 6)

    Returns:
    - compiled Keras model
    """
    time_steps = input_shape[0]
    model = Sequential()

    # Conv 1
    model.add(Conv1D(48, 3, activation='relu', padding='same', input_shape=input_shape))
    if time_steps >= 4:
        model.add(MaxPooling1D(pool_size=2))
        time_steps //= 2

    # Conv 2
    model.add(Conv1D(32, 3, activation='relu', padding='same'))
    if time_steps >= 4:
        model.add(MaxPooling1D(pool_size=2))
        time_steps //= 2

    # Conv 3
    model.add(Conv1D(16, 3, activation='relu', padding='same'))
    if time_steps >= 4:
        model.add(MaxPooling1D(pool_size=2))
        time_steps //= 2

    model.add(Dropout(0.25))

    # LSTM Layers
    model.add(LSTM(20, return_sequences=True))
    model.add(LSTM(20, return_sequences=True))
    model.add(LSTM(20, return_sequences=False))

    model.add(Dropout(0.25))

    # Fully connected
    model.add(Dense(20, activation='relu'))
    model.add(Dense(output_size))  # Output layer (linear)

    model.compile(optimizer=Adam(learning_rate=0.001), loss=MeanAbsoluteError())
    return model


## Training wrapper

In [7]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.losses import MeanAbsoluteError
from tensorflow.keras.metrics import MeanAbsoluteError as MAEMetric
from tensorflow.keras.models import load_model

def train_model(model, X_train, y_train, X_val, y_val, model_path):
    """
    Trains the CNN-LSTM model with ReduceLROnPlateau based on validation MAE.
    Saves the best model based on validation MAE (not loss).
    """
    # Compile with MAE as loss and metric
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss=MeanAbsoluteError(),
                  metrics=[MAEMetric(name='mae')])

    # Learning rate scheduler based on val_mae
    lr_schedule = ReduceLROnPlateau(
        monitor='val_loss',
        patience=10,
        factor=0.8,
        min_lr=1e-5,
        verbose=1
    )

    # Save best model based on val_mae
    checkpoint = ModelCheckpoint(
        model_path,
        monitor='val_mae',
        save_best_only=True,
        save_weights_only=False,
        verbose=1
    )

    # Train
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=150,
        batch_size=128,
        callbacks=[lr_schedule, checkpoint],
        verbose=1
    )

    print(history.history.keys())
    return model, history


## Evaluation

In [5]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

def evaluate_forecasting_model(model, X_test, y_test, scaler):
    """
    Evaluates a forecasting model and prints MAE, RMSE, MAPE.
    
    Parameters:
    - model: trained Keras model
    - X_test: test input features
    - y_test: true test values (scaled or unscaled)
    - scaler: fitted MinMaxScaler used for target (optional, for inverse scaling)
    
    Returns:
    - Dictionary of MAE, RMSE, MAPE
    """
    # Predict
    y_pred = model.predict(X_test)

    # Inverse scale if scaler is given
    if scaler is not None:
        y_pred = scaler.inverse_transform(y_pred)
        y_test = scaler.inverse_transform(y_test)

    # Evaluation metrics
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mape = np.mean(np.abs((y_test - y_pred) / (y_test + 1e-8))) * 100

    print(f"📊 Evaluation Metrics:")
    print(f"MAE:  {mae:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAPE: {mape:.2f}%")

    return {"mae": mae, "rmse": rmse, "mape": mape}


## Main loop

In [8]:
import pandas as pd
from tensorflow.keras.models import load_model
import os 

results = []

timesteps_list = [2, 6, 12]
horizon_list = [1, 2, 6]

for input_window in timesteps_list:
    for forecast_horizon in horizon_list:
        print(f"\n🔁 Training with lookback={input_window}, horizon={forecast_horizon}")
        try:
            # Prepare sequences
            X_train, y_train, X_val, y_val, X_test, y_test, scaler = prepare_sequences(
                df, input_window=input_window, forecast_horizon=forecast_horizon,
                val_ratio=0.2, test_ratio=0.1
            )

            # Build model
            model = build_cnn_lstm_model(
                input_shape=(X_train.shape[1], X_train.shape[2]),
                output_size=y_train.shape[1]
            )

            model_path = f"best_model_{input_window}_{forecast_horizon}.h5"

            # Train and save the best model
            model, history = train_model(model, X_train, y_train, X_val, y_val, model_path=model_path)

            # Try to load the best saved model; fallback to last epoch model if not saved
            if os.path.exists(model_path):
                print(f"✅ Loaded saved model from: {model_path}")
                best_model = load_model(model_path)
            else:
                print(f"⚠️ No saved model found for {input_window}-{forecast_horizon}. Using model from last epoch.")
                best_model = model

            # Evaluate loaded model
            metrics = evaluate_forecasting_model(best_model, X_test, y_test, scaler)

            # Save results
            results.append({
                "lookback": input_window,
                "lookahead": forecast_horizon,
                "mae": metrics["mae"],
                "rmse": metrics["rmse"],
                "mape": metrics["mape"]
            })
        
        except Exception as e:
            print(f"❌ Failed for lookback={input_window}, horizon={forecast_horizon}: {e}")
            results.append({
                "lookback": input_window,
                "lookahead": forecast_horizon,
                "mae": None,
                "rmse": None,
                "mape": None
            })

# Convert to DataFrame
results_df = pd.DataFrame(results)
print("\n✅ Final Results:")
print(results_df)



🔁 Training with lookback=2, horizon=1
Epoch 1/150


c:\Users\janav\.pyenv\pyenv-win\versions\3.10.6\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


324/332 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0511 - mae: 0.0511
Epoch 1: val_mae improved from inf to 0.01788, saving model to best_model_2_1.h5


332/332 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0506 - mae: 0.0506 - val_loss: 0.0179 - val_mae: 0.0179 - learning_rate: 0.0010
Epoch 2/150
322/332 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0194 - mae: 0.0194
Epoch 2: val_mae improved from 0.01788 to 0.01450, saving model to best_model_2_1.h5


332/332 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0193 - mae: 0.0193 - val_loss: 0.0145 - val_mae: 0.0145 - learning_rate: 0.0010
Epoch 3/150
323/332 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0170 - mae: 0.0170
Epoch 3: val_mae did not improve from 0.01450
332/332 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0170 - mae: 0.0170 - val_loss: 0.0147 - val_mae: 0.0147 - learning_rate: 0.0010
Epoch 4/150
329/332 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0165 - mae: 0.0165
Epoch 4: val_mae did not improve from 0.01450
332/332 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0165 - mae: 0.0165 - val_loss: 0.0169 - val_mae: 0.0169 - learning_rate: 0.0010
Epoch 5/150
330/332 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0165 - mae: 0.0165
Epoch 5: val_mae did not improve from 0.01450
332/332 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0165 - mae: 0.0165 - val_loss: 0.0212 - val_mae: 0.0212 - learning_rate: 0.0010
Epoch 6/150
326/332 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0157 - mae: 0.0157
Epoch 6: v

...

331/331 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0247 - mae: 0.0247 - val_loss: 0.0245 - val_mae: 0.0245 - learning_rate: 0.0010
Epoch 7/150
326/331 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0237 - mae: 0.0237
Epoch 7: val_mae improved from 0.02447 to 0.02340, saving model to best_model_12_6.h5


331/331 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0237 - mae: 0.0237 - val_loss: 0.0234 - val_mae: 0.0234 - learning_rate: 0.0010
Epoch 8/150
327/331 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0231 - mae: 0.0231
Epoch 8: val_mae did not improve from 0.02340
331/331 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0231 - mae: 0.0231 - val_loss: 0.0239 - val_mae: 0.0239 - learning_rate: 0.0010
Epoch 9/150
326/331 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0226 - mae: 0.0226
Epoch 9: val_mae did not improve from 0.02340
331/331 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0226 - mae: 0.0226 - val_loss: 0.0236 - val_mae: 0.0236 - learning_rate: 0.0010
Epoch 10/150
326/331 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0221 - mae: 0.0221
Epoch 10: val_mae did not improve from 0.02340
331/331 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0221 - mae: 0.0221 - val_loss: 0.0240 - val_mae: 0.0240 - learning_rate: 0.0010
Epoch 11/150
324/331 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0218 - mae: 0.0218
Epoch 1

331/331 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0215 - mae: 0.0215 - val_loss: 0.0232 - val_mae: 0.0232 - learning_rate: 0.0010
Epoch 15/150
325/331 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0210 - mae: 0.0210
Epoch 15: val_mae improved from 0.02322 to 0.02310, saving model to best_model_12_6.h5


331/331 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0210 - mae: 0.0210 - val_loss: 0.0231 - val_mae: 0.0231 - learning_rate: 0.0010
Epoch 16/150
325/331 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0212 - mae: 0.0212
Epoch 16: val_mae did not improve from 0.02310
331/331 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0212 - mae: 0.0212 - val_loss: 0.0234 - val_mae: 0.0234 - learning_rate: 0.0010
Epoch 17/150
326/331 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0208 - mae: 0.0208
Epoch 17: val_mae did not improve from 0.02310
331/331 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0208 - mae: 0.0208 - val_loss: 0.0242 - val_mae: 0.0242 - learning_rate: 0.0010
Epoch 18/150
323/331 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0207 - mae: 0.0207
Epoch 18: val_mae did not improve from 0.02310
331/331 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0207 - mae: 0.0207 - val_loss: 0.0237 - val_mae: 0.0237 - learning_rate: 0.0010
Epoch 19/150
327/331 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0205 - mae: 0.0205
Epo

dict_keys(['loss', 'mae', 'val_loss', 'val_mae', 'learning_rate'])
✅ Loaded saved model from: best_model_12_6.h5
190/190 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
📊 Evaluation Metrics:
MAE:  0.9104
RMSE: 1.4310
MAPE: 36.88%

✅ Final Results:
   lookback  lookahead       mae      rmse       mape
0         2          1  0.527333  0.905622  22.382654
1         2          2  0.612271  1.052628  26.539233
2         2          6  0.875939  1.412999  36.997665
3         6          1  0.473922  0.816099  21.056187
4         6          2  0.641234  1.051489  26.998993
5         6          6  0.876572  1.402155  37.825529
6        12          1  0.545471  0.889349  23.492209
7        12          2  0.633103  1.039474  25.970486
8        12          6  0.910429  1.430973  36.882742


In [9]:
results_df.to_csv("results.csv", index=False)